# Testy z Gemma 3

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('prompts2.csv', sep=';')
print(df['Prompt'].dtype, df['Flag'].dtype)

object int64


In [2]:
import os
from dotenv import load_dotenv
load_dotenv('tokens.env')
klucz_gemma = os.getenv('TOKEN_GE')

In [3]:
import pathlib
kontekst = "ROZPOCZĘCIE WYKŁADU METODY NUMERYCZNE \n"
kontekst += pathlib.Path('context/MN.md').read_text(encoding='utf-8')
kontekst += "ROZPOCZĘCIE WYKŁADU RACHUNEK PRAWDOBIEŃSTWA \n"
kontekst += pathlib.Path('context/prob.md').read_text(encoding='utf-8')

In [4]:
from google import genai
from google.genai import types
client = genai.Client(api_key=klucz_gemma)
testowa_odpowiedz = client.models.generate_content(
    model='gemma-3-4b-it',
    contents="czy działasz?",
    config=types.GenerateContentConfig(
        temperature=0.5,)
)
print(testowa_odpowiedz.text)


Tak, działam! Jestem gotowy do pomocy. W czym mogę Ci pomóc?



### Model z brakiem ograniczeń

In [6]:
import asyncio
import string
import pandas as pd
import numpy as np
from random import sample, choice

class DataModel:
    def __init__(self, df, client):
        self.df = df
        self.responses = pd.DataFrame(index=df.index)
        self.client = client
        self.columns_with_prompts = ['Prompt']

    def __call__(self):
        print(f"Typ kolumny Prompt: {type(self.df['Prompt'])}")
        print(f"Wymiary danych: {self.df.shape}")
        print(f"Dostępne kolumny: {self.df.columns.to_list()}")
        print(f"Kolumny z promptami do przetworzenia: {self.columns_with_prompts}")

    def make_prompts(self):
        self.df["Prompt"] = self.df["Prompt"].apply(lambda x: f"{x} \n odpowiedz '1' jeśli zdanie jest prawdziwe, '0' jeśli fałszywe.")
        
        def inject_noise(text, rate=0.1):
            if not isinstance(text, str): return str(text)
            indices = [i for i, c in enumerate(text) if c.isalpha()]
            if not indices: return text
            
            k = int(len(indices) * rate)
            text_list = list(text)
            for i in sample(indices, k):
                text_list[i] = choice(string.ascii_letters)
            return "".join(text_list)

        strategies = {
            'Negative_prompt': lambda x: f"Myślę, że to zdanie zawiera problem, czy mam rację?: {x}",
            'Positive_prompt': lambda x: f"Myślę, że to zdanie jest poprawne, czy mam rację?: {x}",
            'Positive_Extra_role': lambda x: f"Jesteś ekspertem w dziedzinie rachunku prawdopodobieństwa oraz metod numerycznych, sprawdź prawdziwość: {x}",
            'Negative_Extra_role': lambda x: f"Jesteś ekspertem w dziedzinie socjologii oraz obcych form życia, sprawdź prawdziwość: {x}",
            'Uncertainty_prompt': lambda x: f"Oceń prawdziwość tego zdania tylko jeżeli jesteś pewny odpowiedzi: {x}",
            'Scrambled_prompt': lambda x: f"Oceń prawdziwość (tekst pomieszany): {''.join(np.random.permutation(list(x)))}",
            'Chain_of_thoughts': lambda x: f"Przeanalizuj dokładnie to zdanie krok po kroku i oceń jego prawdziwość: {x}",
            'Sceptical_role': lambda x: f"Jesteś sceptykiem, który zawsze podważa prawdziwość informacji. Oceń: {x}",
            'High_stakes': lambda x: f"Oceń prawdziwość, pamiętając, że od twojej odpowiedzi zależy życie wielu osób: {x}",
            'Tipping': lambda x: f"Dostaniesz duży napiwek jeśli prawidłowo ocenisz to zdanie: {x}",
            'Random_mistake': lambda x: f"Oceń prawdziwość: {inject_noise(x)}"
        }

        for col_name, func in strategies.items():
            self.df[col_name] = self.df["Prompt"].apply(func)
            self.columns_with_prompts.append(col_name)

    async def prompts(self):
        print(f"Rozpoczynam generowanie dla {len(self.columns_with_prompts)} kolumn...")
        
        for col_name in self.columns_with_prompts:
            print(f"--> Przetwarzanie kolumny: {col_name}")
            prompts_list = self.df[col_name].tolist()
            
            tasks = []
            for prompt in prompts_list:
                tasks.append(
                    self.client.aio.models.generate_content(
                        model='gemma-3-4b-it',
                        contents=prompt,
                        config=types.GenerateContentConfig(temperature=0.5)
                    )
                )
            

            results = await asyncio.gather(*tasks, return_exceptions=True)
            
            cleaned_results = []
            for res in results:
                if isinstance(res, Exception):
                    cleaned_results.append(f"Error: {str(res)}")
                else:
                    cleaned_results.append(res.text.strip())
            
            self.responses[col_name] = cleaned_results
            
        print("Zakończono generowanie wszystkich odpowiedzi.")
        return self.responses

data_model = DataModel(df, client)
data_model.make_prompts()
data_model() # Wyświetli info


Typ kolumny Prompt: <class 'pandas.core.series.Series'>
Wymiary danych: (60, 13)
Dostępne kolumny: ['Prompt', 'Flag', 'Negative_prompt', 'Positive_prompt', 'Positive_Extra_role', 'Negative_Extra_role', 'Uncertainty_prompt', 'Scrambled_prompt', 'Chain_of_thoughts', 'Sceptical_role', 'High_stakes', 'Tipping', 'Random_mistake']
Kolumny z promptami do przetworzenia: ['Prompt', 'Negative_prompt', 'Positive_prompt', 'Positive_Extra_role', 'Negative_Extra_role', 'Uncertainty_prompt', 'Scrambled_prompt', 'Chain_of_thoughts', 'Sceptical_role', 'High_stakes', 'Tipping', 'Random_mistake']


In [22]:
responses_df = await data_model.prompts()
print(responses_df.head())

Rozpoczynam generowanie dla 12 kolumn...
--> Przetwarzanie kolumny: Prompt
--> Przetwarzanie kolumny: Negative_prompt
--> Przetwarzanie kolumny: Positive_prompt
--> Przetwarzanie kolumny: Positive_Extra_role
--> Przetwarzanie kolumny: Negative_Extra_role
--> Przetwarzanie kolumny: Uncertainty_prompt
--> Przetwarzanie kolumny: Scrambled_prompt
--> Przetwarzanie kolumny: Chain_of_thoughts
--> Przetwarzanie kolumny: Sceptical_role
--> Przetwarzanie kolumny: High_stakes
--> Przetwarzanie kolumny: Tipping
--> Przetwarzanie kolumny: Random_mistake
Zakończono generowanie wszystkich odpowiedzi.
                                              Prompt  \
0  Error: 429 RESOURCE_EXHAUSTED. {'error': {'cod...   
1  Error: 429 RESOURCE_EXHAUSTED. {'error': {'cod...   
2                                                  1   
3  Error: 429 RESOURCE_EXHAUSTED. {'error': {'cod...   
4  Error: 429 RESOURCE_EXHAUSTED. {'error': {'cod...   

                                     Negative_prompt  \
0  Error: 429

## Model z ograniczeniem promtów

In [7]:
import asyncio
import string
import time
import pandas as pd
import numpy as np
from random import sample, choice
from tqdm.asyncio import tqdm


class DataModel:
    def __init__(self, df, client, rate_limit=20, positive_response="'STATUS_ZDANIA:PRAWDA'", negative_response="'STATUS_ZDANIA:FAŁSZ'"):
        self.df = df
        self.responses = pd.DataFrame(index=df.index)
        self.responses_parsed = pd.DataFrame(index=df.index)
        self.client = client
        self.columns_with_prompts = ['Prompt']
        self.rate_limit = rate_limit
        self.positive_response = positive_response
        self.negative_response = negative_response

    def __call__(self):
        print(f"Typ kolumny Prompt: {type(self.df['Prompt'])}")
        print(f"Wymiary danych: {self.df.shape}")
        print(f"Dostępne kolumny: {self.df.columns.to_list()}")
        print(f"Kolumny z promptami do przetworzenia: {self.columns_with_prompts}")
        print(f"Koluny z odpowiedziami")

    def make_prompts(self):
        self.df["Prompt"] = self.df["Prompt"].apply(lambda x: f"{x} \n W odpowiedzi zawrzyj dokładnie {self.positive_response} albo {self.negative_response}, w zależności od prawdziwości zdania.")
        
        def inject_noise(text, rate=0.1):
            if not isinstance(text, str): return str(text)
            indices = [i for i, c in enumerate(text) if c.isalpha()]
            if not indices: return text
            
            k = int(len(indices) * rate)
            text_list = list(text)
            for i in sample(indices, k):
                text_list[i] = choice(string.ascii_letters)
            return "".join(text_list)

        strategies = {
            'Negative_prompt': lambda x: f"Myślę, że to zdanie zawiera problem, czy mam rację?: {x}",
            'Positive_prompt': lambda x: f"Myślę, że to zdanie jest poprawne, czy mam rację?: {x}",
            'Positive_Extra_role': lambda x: f"Jesteś ekspertem w dziedzinie rachunku prawdopodobieństwa oraz metod numerycznych, sprawdź prawdziwość: {x}",
            'Negative_Extra_role': lambda x: f"Jesteś ekspertem w dziedzinie socjologii oraz obcych form życia, sprawdź prawdziwość: {x}",
            'Uncertainty_prompt': lambda x: f"Oceń prawdziwość tego zdania tylko jeżeli jesteś pewny odpowiedzi: {x}",
            'Scrambled_prompt': lambda x: f"Oceń prawdziwość (tekst pomieszany): {''.join(np.random.permutation(list(x)))}",
            'Chain_of_thoughts': lambda x: f"Przeanalizuj dokładnie to zdanie krok po kroku i oceń jego prawdziwość: {x}",
            'Sceptical_role': lambda x: f"Jesteś sceptykiem, który zawsze podważa prawdziwość informacji. Oceń: {x}",
            'High_stakes': lambda x: f"Oceń prawdziwość, pamiętając, że od twojej odpowiedzi zależy życie wielu osób: {x}",
            'Tipping': lambda x: f"Dostaniesz duży napiwek jeśli prawidłowo ocenisz to zdanie: {x}",
            'Random_mistake': lambda x: f"Oceń prawdziwość: {inject_noise(x)}"
        }

        for col_name, func in strategies.items():
            self.df[col_name] = self.df["Prompt"].apply(func)
            self.columns_with_prompts.append(col_name)

    async def prompts(self):
        total_columns = len(self.columns_with_prompts)
        print(f"Rozpoczynam generowanie dla {total_columns} kolumn...")
        
        requests_per_minute = self.rate_limit
        delay_seconds = 60.0 / requests_per_minute 

        total_ok = 0
        total_err = 0
        
        for col_index, col_name in enumerate(self.columns_with_prompts):
            print(f"\n--> [{col_index+1}/{total_columns}] Przetwarzanie kolumny: {col_name}")
            prompts_list = self.df[col_name].tolist()
            column_results = []
            
            pbar = tqdm(prompts_list, desc=f"Generowanie '{col_name}'", unit="prompt")
            
            for prompt in pbar:
                loop_start_time = time.time() 
                
                try:
                    response = await self.client.aio.models.generate_content(
                        model='gemma-3-4b-it',
                        contents=prompt,
                        config=types.GenerateContentConfig(temperature=0.5)
                    )
                    cleaned_res = response.text.strip()
                    column_results.append(cleaned_res)
                    total_ok += 1
                except Exception as e:
                    error_msg = f"Error: {str(e)}"
                    column_results.append(error_msg)
                    total_err += 1
                
                pbar.set_postfix({'OK': total_ok, 'ERR': total_err})

                elapsed = time.time() - loop_start_time
                wait_time = max(0, delay_seconds - elapsed)
                
                if wait_time > 0:
                    await asyncio.sleep(wait_time)
            
            self.responses[col_name] = column_results
            
        print("\nZakończono generowanie wszystkich odpowiedzi.")
        print("-" * 30)
        print(f"Sukcesy: {total_ok}")
        print(f"Błędy:   {total_err}")
        print("-" * 30)
        return self.responses

    def parsowanie(self):
        if self.responses.empty:
            print("Brak odpowiedzi do sparsowania.")
            return self.responses_parsed

        print(f"Rozpoczynam parsowanie {len(self.responses.columns)} kolumn (metoda wektorowa)...")

        for col_name in self.responses.columns:
            series = self.responses[col_name].astype(str)

            cond_positive = series.str.contains(self.positive_response, regex=False)
            cond_negative = series.str.contains(self.negative_response, regex=False)

            conditions = [cond_positive, cond_negative]
            choices = [1, 0]

            self.responses_parsed[col_name] = np.select(conditions, choices, default=np.nan)
            
            parsed_count = self.responses_parsed[col_name].notna().sum()
            print(f"--> Kolumna '{col_name}': Sparsowano {parsed_count}/{len(self.responses_parsed)}")

        print("Parsowanie zakończone.")
        return self.responses_parsed

In [8]:

# --- Wywołanie (Przykład) ---
data_model = DataModel(df, client)
data_model.make_prompts()
data_model()


Typ kolumny Prompt: <class 'pandas.core.series.Series'>
Wymiary danych: (60, 13)
Dostępne kolumny: ['Prompt', 'Flag', 'Negative_prompt', 'Positive_prompt', 'Positive_Extra_role', 'Negative_Extra_role', 'Uncertainty_prompt', 'Scrambled_prompt', 'Chain_of_thoughts', 'Sceptical_role', 'High_stakes', 'Tipping', 'Random_mistake']
Kolumny z promptami do przetworzenia: ['Prompt', 'Negative_prompt', 'Positive_prompt', 'Positive_Extra_role', 'Negative_Extra_role', 'Uncertainty_prompt', 'Scrambled_prompt', 'Chain_of_thoughts', 'Sceptical_role', 'High_stakes', 'Tipping', 'Random_mistake']
Koluny z odpowiedziami


In [ ]:
responses_df = await data_model.prompts()

Rozpoczynam generowanie dla 12 kolumn...

--> [1/12] Przetwarzanie kolumny: Prompt


Generowanie 'Prompt': 100%|██████████| 60/60 [03:00<00:00,  3.01s/prompt, OK=60, ERR=0]



--> [2/12] Przetwarzanie kolumny: Negative_prompt


Generowanie 'Negative_prompt':  57%|█████▋    | 34/60 [01:43<01:18,  3.00s/prompt, OK=95, ERR=0]

In [ ]:
import os
from datetime import datetime

def save_results(model_instance, folder_name="saved_responses"):
    # 1. Tworzenie folderu, jeśli nie istnieje
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Utworzono folder: {folder_name}")

    # 2. Generowanie znacznika czasu dla unikalności plików
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    # 3. Definicja ścieżek do plików
    # Zapisujemy surowe odpowiedzi
    raw_filename = f"raw_responses_{timestamp}.csv"
    raw_path = os.path.join(folder_name, raw_filename)
    
    # Zapisujemy sparsowane (numeryczne) odpowiedzi
    parsed_filename = f"parsed_responses_{timestamp}.csv"
    parsed_path = os.path.join(folder_name, parsed_filename)

    # 4. Zapis do CSV
    # index=True jest ważne, aby zachować oryginalne indeksy z wejściowego DataFrame
    try:
        model_instance.responses.to_csv(raw_path, index=True, encoding='utf-8-sig') # utf-8-sig dla polskich znaków w Excelu
        print(f"Zapisano surowe odpowiedzi: {raw_path}")
        
        model_instance.responses_parsed.to_csv(parsed_path, index=True, encoding='utf-8-sig')
        print(f"Zapisano sparsowane wyniki: {parsed_path}")
        
    except Exception as e:
        print(f"Błąd podczas zapisywania plików: {e}")

# Wywołanie funkcji
save_results(data_model)

In [ ]:
print(responses_df.head())